In [ ]:
from sklearn.model_selection import train_test_split
from tensorflow.keras.utils import to_categorical
import cv2
import numpy as np
import os
from matplotlib import pyplot as plt
import time
import mediapipe as mp
import re
import random
import tensorflow as tf


# Load dataset landmarks mediapipe

In [ ]:
sequence_length = 10 # quantidade máxima de frames

def natural_sort_key(s, _nsre=re.compile('([0-9]+)')):
    return [
        int(text)
        if text.isdigit() else text.lower()
        for text in _nsre.split(s)]


def landmarks_data(DATA_PATH):
    sequences, labels = [], []
    actions = np.array(sorted(os.listdir(DATA_PATH)))
    label_map = {label:num for num, label in enumerate(actions)}
    for action in actions:
        for qnt_videos in os.listdir(os.path.join(DATA_PATH,action)):
            if (len(os.listdir(DATA_PATH + '/' + action + '/' + qnt_videos))) < sequence_length:
                continue
            window = []

            a = random.sample(os.listdir(DATA_PATH + '/' + action + '/' + qnt_videos), sequence_length)
            for frame_num in sorted(a, key=natural_sort_key):
                res = np.load(os.path.join(DATA_PATH, action, str(qnt_videos), "{}".format(frame_num)))
                window.append(res)
            sequences.append(window)
            labels.append(label_map[action])
    return sequences, labels




In [ ]:
path = 'Landmarks_videos' # pasta de entrada
data, labels = landmarks_data(path) 

# train test split

In [ ]:
classes = len(set(labels)
y = to_categorical(labels).astype(int)
x_train, x_test, y_train, y_test = train_test_split(data, y, test_size=0.2)


# Lstm model

In [ ]:
model = Sequential()
model.add(LSTM(128, return_sequences=True, activation='relu',input_shape=(np.shape(data)[1], np.shape(data)[2])))  # (qtd_frames, qtd_landmarks)
model.add(LSTM(64, return_sequences=True, activation='relu'))  
model.add(LSTM(32, activation='relu'))  
model.add(Dense(classes, activation='softmax'))

model.compile(loss='categorical_crossentropy',
              optimizer='sgd',
              metrics=['accuracy'])

epochs=100
model.fit(x_train, y_train, epochs=epochs, validation_split = 0.1)

# Accuracy test

In [ ]:
from sklearn.metrics import classification_report, accuracy_score
from sklearn.metrics import confusion_matrix

predict = model.predict(np.array(x_test))
y_pred = np.argmax(predict, axis = 1)
y_true = np.argmax(y_test, axis = 1)

print("Acurácia:", accuracy_score(y_true, y_pred))
print(classification_report(y_true, y_pred))
